In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import os
import math

In [2]:
DATA_BASE_PATH = 'DATA'
GENERATOR = '1'
RESULTS_BASE_PATH = 'RESULTS'

NODES = [(5, 8), (8, 5), (5, 12), (8, 12)]

# FAILUER_TIME = 16000

In [3]:
def load_sequence(file):
    Y = np.load(os.path.join(DATA_BASE_PATH, GENERATOR, f"{file}.npy"))
    Y = Y.reshape((-1))

    return Y

def split_sequence(sequence, n_steps_in, n_steps_out):
    X = [sequence[i:i+n_steps_in]
         for i in range(len(sequence) - (n_steps_in + n_steps_out) + 1)]
    y = [sequence[i:i+n_steps_out]
         for i in range(n_steps_in, len(sequence) - n_steps_out + 1)]

    return np.array(X).reshape(len(X), n_steps_in, 1), np.array(y)

def define_model(n_steps_in, n_steps_out) -> Sequential:
    model = Sequential()
    model.add(LSTM(10, activation='relu', input_shape=(n_steps_in, 1)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    return model

def run_experiment(exp, file, n_steps_in, n_steps_out, train_size, epochs=5, chunk_size=0):
    sequence = load_sequence(file)
    X, y = split_sequence(sequence, n_steps_in, n_steps_out)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42, shuffle=False)
    model = define_model(n_steps_in, n_steps_out)
    model.fit(X_train, y_train, epochs=epochs)

    if(chunk_size):
        
        chunks_number = math.floor(X_test.shape[0] / chunk_size)
        max_samples = chunks_number * chunk_size
        y_test = y_test[:max_samples]
        X_test = X_test[:max_samples]

        y_predicted = np.zeros(y_test.shape)

        X_test_chunks = np.array(np.array_split(X_test, chunks_number))
        y_test_chunks = np.array(np.array_split(y_test, chunks_number))

        for i in range(chunks_number):
            y_predicted[i * chunk_size : (i + 1) * chunk_size] = model.predict(X_test_chunks[i])
            model.fit(X_test_chunks[i], y_test_chunks[i], epochs)

    else:
        y_predicted = model.predict(X_test)

    exp_path = os.path.join(RESULTS_BASE_PATH, exp)

    if not os.path.exists(exp_path):
        os.makedirs(exp_path)

    np.save(os.path.join(exp_path, "true.npy"), y_test)
    np.save(os.path.join(exp_path, "predicted.npy"), y_predicted)

def run_all_experiments(exp, n_steps_in, n_steps_out, train_size, epochs=5, chunk_size=0):
    for n_from, n_to in NODES:
        run_experiment(f'{exp}___{n_from}to{n_to}', f'Y_{n_from}_{n_to}', n_steps_in, n_steps_out, train_size, epochs, chunk_size)

    

In [4]:
# run_all_experiments('LSTM_5_in_1_out', 5, 1, 10000, 5)
run_all_experiments('LSTM_1_CHUNK_5_in_3_out', 5, 3, 10000, 10, 100)


Epoch 1/10
313/313 [==============================] - 4s 3ms/step - loss: 121248.0234
Epoch 2/10
313/313 [==============================] - 1s 2ms/step - loss: 14867.3828A: 0s - loss: 14534.
Epoch 3/10
313/313 [==============================] - 1s 2ms/step - loss: 14698.7520
Epoch 4/10
313/313 [==============================] - 1s 2ms/step - loss: 14328.2676
Epoch 5/10
313/313 [==============================] - 1s 2ms/step - loss: 14161.9453
Epoch 6/10
313/313 [==============================] - 1s 3ms/step - loss: 13783.5693A: 0s - loss: 1
Epoch 7/10
313/313 [==============================] - 1s 4ms/step - loss: 13302.4404
Epoch 8/10
313/313 [==============================] - 1s 4ms/step - loss: 13286.5059
Epoch 9/10
313/313 [==============================] - 1s 4ms/step - loss: 13827.3506A: 1
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 13230.4873
Epoch 1/10
313/313 [==============================] - 3s 3ms/step - loss: 241967.1562
Epoch 2/10
313/313 [=====